In [1]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 1

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
#tf.random.set_seed(seed_value)
# for later versions: 
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
#session_conf = tf.ConfigProto()
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.applications.vgg16 import VGG16
from keras.layers import LSTM
#import numpy as np
import glob
from imageio import imread
from skimage.transform import resize as imresize
from keras.layers import LeakyReLU
import pandas as pd
import numpy as np
seq = 120

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [118]:
train = pd.read_csv('train_zero_padded.csv')
val = pd.read_csv('validation_zero_padded_6.csv')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,2054,2060,2061,2065,2066,2067,2068) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [119]:
val.drop('labels',axis=1,inplace=True)
val.drop('t',axis=1,inplace=True)

In [120]:
val.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'title_x', '0', '1', '2', '3', '4', '5',
       '6',
       ...
       'tags', 'UHID', 'Video (Title)',
       'Total Duration of video file (min.sec)', 'From (min.sec)',
       'To (min.sec)', 'Procedure or manipulation (Name)', 'Label', 'title_y',
       'time'],
      dtype='object', length=2068)

In [49]:
train.columns

Index(['Unnamed: 0', 'title', '0', '1', '2', '3', '4', '5', '6', '7',
       ...
       '2047', 'diaper_change', 'feeding', 'patting', 'uhid', 'Total_frames',
       'frame_number', 'series', 'tags', 'labels'],
      dtype='object', length=2059)

In [6]:
model = Sequential()
#model.add(LSTM(256,dropout=0.2,input_shape=(None,2048)))
model.add(LSTM(512,return_sequences=True,activation='tanh',input_shape=(None,2048)))
#model.add(activation='tanh')
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(256,return_sequences=False))
#model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
es =   EarlyStopping(monitor='val_loss', patience=8, verbose=0)

In [121]:
#train.drop('Unnamed: 0',axis=1,inplace=True)
#train.drop('tags',axis=1,inplace=True)
val.drop('Unnamed: 0',axis=1,inplace=True)
val.drop('Unnamed: 0.1',axis=1,inplace=True)
val.drop('tags',axis=1,inplace=True)

In [122]:
ids = train['uhid'].unique()
ids_val = val['uhid'].unique()

In [123]:
X_val = pd.DataFrame()
X = pd.DataFrame()


In [10]:
for i in ids:
    patient = train[train['uhid']==i]
    total_frames = patient['Total_frames'].unique()
    for j in total_frames:
        x = patient[patient['Total_frames']==j]
        x = x.sort_values(by=['series','frame_number'])             
        x['diaper_change'].fillna(method='ffill',inplace=True)
        x['feeding'].fillna(method='ffill',inplace=True)
        x['patting'].fillna(method='ffill',inplace=True)
        X = X.append(x,ignore_index=True)
        

In [124]:
for i in ids_val:
    patient = val[val['uhid']==i]
    total_frames = patient['Total_frames'].unique()
    for j in total_frames:
        x = patient[patient['Total_frames']==j]
        x = x.sort_values(by=['series','frame_number'])             
        x['diaper_change'].fillna(method='ffill',inplace=True)
        x['feeding'].fillna(method='ffill',inplace=True)
        x['patting'].fillna(method='ffill',inplace=True)
        
        X_val = X_val.append(x,ignore_index=True)

In [12]:
X.drop('title',axis=1,inplace=True)
X.drop('uhid',axis=1,inplace=True)
X.drop('Total_frames',axis=1,inplace=True)
X.drop('frame_number',axis=1,inplace=True)
X.drop('series',axis=1,inplace=True)
X.drop('labels',axis=1,inplace=True)

In [125]:
#X_val.drop('title_x',axis=1,inplace=True)
X_val.drop('uhid',axis=1,inplace=True)
X_val.drop('Total_frames',axis=1,inplace=True)
X_val.drop('frame_number',axis=1,inplace=True)
X_val.drop('series',axis=1,inplace=True)
X_val.drop('title_y',axis=1,inplace=True)
X_val.drop('To (min.sec)',axis=1,inplace=True)
X_val.drop('Procedure or manipulation (Name)',axis=1,inplace=True)
X_val.drop('From (min.sec)',axis=1,inplace=True)
X_val.drop('Total Duration of video file (min.sec)',axis=1,inplace=True)
X_val.drop('Video (Title)',axis=1,inplace=True)
#X_val.drop('Total Duration of video file (min.sec)',axis=1,inplace=True)


In [126]:
X_val.drop('UHID',axis=1,inplace=True)
X_val.drop('Label',axis=1,inplace=True)



In [127]:
X_val.drop('time',axis=1,inplace=True)
X_val.drop('title_x',axis=1,inplace=True)
#X_val.drop('Total Duration of video file (min.sec)',axis=1,inplace=True)

In [128]:
X_val

,0,1,2,3,4,5,6,7,8,9,...,2041,2042,2043,2044,2045,2046,2047,diaper_change,feeding,patting
0,-1.128828,-0.016527,-0.235906,0.698256,0.543091,-0.188442,-1.163451,0.662182,0.054379,-0.649465,...,-0.626992,1.508141,-0.575721,0.488262,0.645725,-0.073558,0.912954,1.0,0.0,0.0
1,-1.018054,0.201727,-0.318205,0.681947,0.933839,-1.113128,-0.237611,-0.608168,0.940666,-0.139904,...,0.670192,0.204470,-0.488212,-0.334271,0.508915,0.046070,0.630174,1.0,0.0,0.0
2,-0.818614,-0.363673,-0.230043,-0.020318,1.628263,-1.709915,0.289006,-0.589129,1.616877,-0.961010,...,1.004921,0.592309,-1.217791,-0.779120,0.815189,-0.300531,0.240677,1.0,0.0,0.0
3,-0.122941,0.333826,-0.083531,-0.085607,1.041494,1.124528,1.176074,0.025734,-0.129343,-0.435462,...,-0.247903,0.946195,-0.367519,0.633486,0.268009,0.680456,1.363238,1.0,0.0,0.0
4,-0.440540,-0.381134,-0.318574,0.613498,1.257364,-0.382675,-0.310298,0.509424,-0.039889,-1.386820,...,-0.095711,2.051557,-0.605160,0.373638,-0.413178,1.078297,0.259838,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,-0.428815,0.782702,0.336739,-0.357267,2.255564,0.791485,0.601925,0.569214,-0.444142,-0.574265,...,-0.606285,1.296535,-1.192561,0.905022,-0.285185,1.044787,0.764147,0.0,0.0,1.0
8756,-0.458468,0.341188,-0.527984,0.101789,1.421188,-0.190172,0.497105,0.461110,0.129927,-0.255010,...,-0.931559,0.913477,-1.026275,0.940874,-0.458415,0.756727,0.651880,0.0,0.0,1.0
8757,0.232102,-0.302728,0.415673,0.349968,0.224248,1.327219,0.246681,1.134674,-0.501171,-0.069691,...,-0.399264,0.775659,-0.432012,1.352493,-0.021824,1.243398,0.247189,0.0,0.0,1.0
8758,0.222461,0.263529,-0.219165,-0.681437,2.198796,0.343512,-0.008500,0.647222,0.776262,-0.926901,...,0.036101,1.799631,-0.755566,0.644902,0.644837,1.018173,1.226416,0.0,0.0,1.0


In [16]:
train_labels1 = X[['diaper_change','feeding','patting']]

In [129]:
validation_labels1 = X_val[['diaper_change','feeding','patting']]

In [18]:
train_labels = []
for i in range(0,len(train_labels1),seq):
#print(i)
    train_labels1 = np.array(train_labels1)
    y1 = train_labels1[i:i+seq]
    train_labels.append(y1[-1])

validation_labels = []
for i in range(0,len(validation_labels1),seq):
#print(i)
    validation_labels1 = np.array(validation_labels1)
    y2 = validation_labels1[i:i+seq]
    validation_labels.append(y2[-1])

In [130]:
validation_labels = []
for i in range(0,len(validation_labels1),seq):
#print(i)
    validation_labels1 = np.array(validation_labels1)
    y2 = validation_labels1[i:i+seq]
    validation_labels.append(y2[-1])

In [131]:
#X.drop(['diaper_change','feeding','patting'],axis=1,inplace=True)
X_val.drop(['diaper_change','feeding','patting'],axis=1,inplace=True)

In [132]:
#X = np.array(X)
#X = X.reshape(-1,seq,X.shape[1])
X_val = np.array(X_val)
X_val = X_val.reshape(-1,seq,X_val.shape[1])

In [133]:
#train_labels = np.array(train_labels)
validation_labels = np.array(validation_labels)

In [22]:
model.fit(X,train_labels,batch_size=120,epochs=40,validation_split=0.2,callbacks=[es])


Train on 255 samples, validate on 64 samples
Epoch 1/40
255/255 [==============================] - 6s 23ms/step - loss: 1.0773 - accuracy: 0.5765 - val_loss: 0.8376 - val_accuracy: 0.3281
Epoch 2/40
255/255 [==============================] - 5s 19ms/step - loss: 0.6543 - accuracy: 0.6824 - val_loss: 0.1528 - val_accuracy: 0.9688
Epoch 3/40
255/255 [==============================] - 5s 19ms/step - loss: 0.4337 - accuracy: 0.8784 - val_loss: 0.2024 - val_accuracy: 1.0000
Epoch 4/40
255/255 [==============================] - 5s 19ms/step - loss: 0.3693 - accuracy: 0.9020 - val_loss: 0.1122 - val_accuracy: 1.0000
Epoch 5/40
255/255 [==============================] - 5s 19ms/step - loss: 0.3073 - accuracy: 0.9020 - val_loss: 0.0615 - val_accuracy: 1.0000
Epoch 6/40
255/255 [==============================] - 5s 19ms/step - loss: 0.2370 - accuracy: 0.9216 - val_loss: 0.0711 - val_accuracy: 1.0000
Epoch 7/40
255/255 [==============================] - 5s 19ms/step - loss: 0.2077 - accuracy: 0.

In [134]:
answer = model.predict(X_val)

In [135]:
y_pred = []
for i in answer:
    y_pred.append(np.argmax(i))

In [136]:
test = []

In [137]:
for i in range(0,len(validation_labels1),seq):
    validation_labels1 = np.array(validation_labels1)
    y2 = validation_labels1[i:i+seq]
    print(i,y2[-1])
    test.append(y2[-1])

0 [1. 0. 0.]
120 [1. 0. 0.]
240 [1. 0. 0.]
360 [1. 0. 0.]
480 [1. 0. 0.]
600 [1. 0. 0.]
720 [1. 0. 0.]
840 [1. 0. 0.]
960 [0. 1. 0.]
1080 [0. 1. 0.]
1200 [0. 1. 0.]
1320 [0. 1. 0.]
1440 [0. 1. 0.]
1560 [0. 1. 0.]
1680 [0. 1. 0.]
1800 [0. 1. 0.]
1920 [0. 1. 0.]
2040 [0. 1. 0.]
2160 [0. 1. 0.]
2280 [0. 1. 0.]
2400 [0. 1. 0.]
2520 [0. 1. 0.]
2640 [0. 1. 0.]
2760 [0. 1. 0.]
2880 [0. 0. 1.]
3000 [0. 0. 1.]
3120 [0. 0. 1.]
3240 [0. 0. 1.]
3360 [0. 0. 1.]
3480 [0. 0. 1.]
3600 [0. 0. 1.]
3720 [0. 0. 1.]
3840 [0. 0. 1.]
3960 [0. 0. 1.]
4080 [0. 0. 1.]
4200 [0. 0. 1.]
4320 [0. 0. 1.]
4440 [0. 0. 1.]
4560 [0. 0. 1.]
4680 [0. 0. 1.]
4800 [0. 0. 1.]
4920 [0. 0. 1.]
5040 [0. 0. 1.]
5160 [1. 0. 0.]
5280 [1. 0. 0.]
5400 [0. 0. 1.]
5520 [0. 0. 1.]
5640 [1. 0. 0.]
5760 [1. 0. 0.]
5880 [1. 0. 0.]
6000 [1. 0. 0.]
6120 [0. 0. 1.]
6240 [0. 0. 1.]
6360 [0. 0. 1.]
6480 [0. 0. 1.]
6600 [0. 0. 1.]
6720 [0. 0. 1.]
6840 [0. 0. 1.]
6960 [0. 0. 1.]
7080 [0. 0. 1.]
7200 [0. 0. 1.]
7320 [0. 0. 1.]
7440 [0. 0. 1.]
756

In [138]:
y_tp = []
for i in test:
    y_tp.append(np.argmax(i))


In [139]:
from sklearn.metrics import classification_report,confusion_matrix

In [140]:
print(classification_report(y_tp,y_pred))

              precision    recall  f1-score   support

           0       0.40      0.14      0.21        14
           1       0.48      0.81      0.60        16
           2       0.93      0.88      0.90        43

    accuracy                           0.73        73
   macro avg       0.60      0.61      0.57        73
weighted avg       0.73      0.73      0.71        73



In [141]:
len(test)

73

In [142]:
print(confusion_matrix(y_tp,y_pred))

[[ 2 12  0]
 [ 0 13  3]
 [ 3  2 38]]
